![imagen con dos cerebros enfrentados, el de la izquierda azul y el de la derecha naranja de los que salen ramificaciones](../img/img_cabecera.PNG)

# Análisis y visualizaciones de los datos transformados sobre el comportamiento de personas introvertidas y extrovertidas